## Models

In [53]:
# class NetworkNode:
#     def __init__(self, id) -> None:
#         self.id = id

#     @classmethod
#     def from_dict(cls, data):
#         return cls(**data)

# class NetworkEdge:
#     def __init__(self, source, target, weight) -> None:
#         self.source = source
#         self.target = target
#         self.weight = weight

#     @classmethod
#     def from_dict(cls, data):
#         return cls(**data)

# class NetworkData:
#     def __init__(self, nodes: list[NetworkNode], links: list[NetworkEdge]) -> None:
#         self.nodes = nodes
#         self.links = links

#     @classmethod
#     def from_dict(cls, data):
#         return cls(**data)

## Data Retreival

In [1]:
import json

def load_data():
    with open('./data/data.json', 'r') as file:
        data = json.load(file)

    return data

# import json
# import copy

# data_path = './data/skott.json'
# # data_path = './data/skott-apps.json'

# with open(data_path, 'r') as f:
#     json_data = json.load(f)

# temp_json_data = copy.deepcopy(json_data)
# for key in temp_json_data:
#     if key.startswith('examples/') or key.startswith('docker') or key.startswith('tests') or key.startswith('playwright-tests') or 'config.' in key:
#         del json_data[key]

# data = {
#     'nodes': [],
#     'links': [],
# }

# for key in json_data:
#     data['nodes'].append({
#         'id': key
#     })

#     for adjacent in json_data[key]['adjacentTo']:
#         data['links'].append({
#             'source': key,
#             'target': adjacent,
#             'waeight': 1,
#         })

# print(f"nodes: {len(data['nodes'])}")
# print(f"links: {len(data['links'])}")


# # output_file = "./data.json"

# # with open(output_file, 'w') as f:
# #     json.dump(data, f)

# # print(f"Data saved as JSON file: {output_file}")

## Generate Pyvis Graph

In [3]:
def filter_nodes(
        nodes, 
        starts_with=[], 
        ends_with=[], 
        contains=[], 
        not_starts_with=[],
        not_ends_with=[],
        not_contains=[]
        ):
    filter_nodes = []
    
    for node in nodes:
        node_starts_with = True if len(starts_with) == 0 else False
        node_ends_with = True if len(ends_with) == 0 else False
        node_contains = True if len(contains) == 0 else False
        node_not_starts_with = True if len(not_starts_with) == 0 else False
        node_not_ends_with = True if len(not_ends_with) == 0 else False
        node_not_contains = True if len(not_contains) == 0 else False

        node_path = node['id']

        for start_with in starts_with:
            if node_path.startswith(start_with):
                node_starts_with = True

        if not node_starts_with:
            continue

        for end_with in ends_with:
            if node_path.endswith(end_with):
                node_ends_with = True

        if not node_ends_with:
            continue

        for contain in contains:
            if contain in node_path:
                node_contains = True

        if not node_contains:
            continue

        for not_start_with in not_starts_with:
            if not node_path.startswith(not_start_with):
                node_not_starts_with = True

        if not node_not_starts_with:
            continue

        for not_end_with in not_ends_with:
            if not node_path.endswith(not_end_with):
                node_not_ends_with = True

        if not node_not_ends_with:
            continue

        for not_contain in not_contains:
            if not not_contain in node_path:
                node_not_contains = True

        if not node_not_contains:
            continue

        filter_nodes.append(node)

    return filter_nodes

# nodes = load_data()['nodes']
# filter_nodes(nodes, starts_with=['apps', 'packages'])[-1]

In [56]:
def generate_nodes(nodes, depth, keep_folder_structure=True):
    graph_nodes = []
    graph_links = []

    for node in nodes:
        depth_path = node['id'].split('/')[:depth] if depth > 0 else node['id'].split('/')
        
        if keep_folder_structure:
            previous_path = ''
            
            for path in depth_path:
                id = f'{previous_path}/{path}' if previous_path != '' else path

                if not any(graph_node['id'] == id for graph_node in graph_nodes):
                    graph_nodes.append({
                        'id': id,
                        'label': path,
                    })

                if previous_path != '' and not any(graph_link['source'] == previous_path and graph_link['target'] == id for graph_link in graph_links):
                    graph_links.append({
                        'source': previous_path,
                        'target': id,
                        'weight': 0
                    })
                
                previous_path = id
        else:
            id = '/'.join(depth_path)
            # if not node in graph_nodes:
            if not any(graph_node['id'] == id for graph_node in graph_nodes):
                nodes.append({'id': id})

    return (graph_nodes, graph_links)

# data = load_data()
# nodes = filter_nodes(data['nodes'], starts_with=['apps'])
# generate_nodes(nodes, 2, keep_folder_structure=False)

In [64]:
def generate_edges(nodes, links, depth):
    edges = []

    for link in links:
        source_node_path = '/'.join(link['source'].split('/')[:depth] if depth > 0 else link['source'].split('/'))
        target_node_path = '/'.join(link['target'].split('/')[:depth] if depth > 0 else link['target'].split('/'))

        if source_node_path == target_node_path:
            continue

        if not any(node['id'] == source_node_path for node in nodes):
            continue

        if not any(node['id'] == target_node_path for node in nodes):
            continue

        if any(edge['source'] == source_node_path and edge['target'] == target_node_path for edge in edges):
            for edge in edges: 
                if edge['source'] == source_node_path and edge['target'] == target_node_path:
                    edge['weight'] = edge['weight'] + 1
        else:
            edges.append({'source': source_node_path, 'target': target_node_path, 'weight': 1})

    return edges

# data = load_data()
# nodes = filter_nodes(data['nodes'], starts_with=['apps'])
# (nodes, res_edges) = generate_nodes(nodes, 2)
# res_edges = []
# links = generate_edges(nodes, data['links'], 2)
# print(links)

In [58]:
from pyvis.network import Network

def construct_network_graph(
        nodes,
        edges,
        depth, 
        starts_with=[], 
        ends_with=[], 
        contains=[], 
        not_starts_with=[],
        not_ends_with=[],
        not_contains=[],
        keep_folder_structure=True):    
    nodes = filter_nodes(nodes, starts_with, ends_with, contains, not_starts_with, not_ends_with, not_contains)

    (nodes, links) = generate_nodes(nodes, depth, keep_folder_structure=keep_folder_structure)
    links = links + generate_edges(nodes, edges, depth)

    net = Network(notebook=True, cdn_resources='in_line', select_menu=True, filter_menu=True, directed=True)

    for node in nodes:
        net.add_node(node['id'], label=node['id'])

    for link in links:
        net.add_edge(link['source'], link['target'], label=str(link['weight']) if link['weight'] > 0 else None, title=link['weight'])

    return net.generate_html()

# data = load_data()
# construct_network_graph(data['nodes'], data['links'], 3, starts_with=['apps', 'packages'])

In [59]:
from pyvis.network import Network

def generate_graph(
        data, 
        depth, 
        starts_with=[], 
        ends_with=[], 
        contains=[],  
        not_starts_with=[],
        not_ends_with=[],
        not_contains=[],
        keep_folder_structure=True
        ) -> str:
    nodes = []
    links = []

    for node in data['nodes']:
        node_starts_with = True if len(starts_with) == 0 else False
        node_ends_with = True if len(ends_with) == 0 else False
        node_contains = True if len(contains) == 0 else False
        node_not_starts_with = True if len(not_starts_with) == 0 else False
        node_not_ends_with = True if len(not_ends_with) == 0 else False
        node_not_contains = True if len(not_contains) == 0 else False

        node_path: str = node['id']
        depth_path = node['id'].split('/')[:depth] if depth > 0 else node['id'].split('/')

        for start_with in starts_with:
            if node_path.startswith(start_with):
                node_starts_with = True

        if not node_starts_with:
            continue

        for end_with in ends_with:
            if node_path.endswith(end_with):
                node_ends_with = True

        if not node_ends_with:
            continue

        for contain in contains:
            if contain in node_path:
                node_contains = True

        if not node_contains:
            continue

        # for not_start_with in not_starts_with:
        #     if node_path.startswith(not_start_with):
        #         node_not_starts_with = False

        # if not node_not_starts_with:
        #     continue

        # for not_end_with in not_ends_with:
        #     if node_path.endswith(not_end_with):
        #         node_not_ends_with = False

        # if node_not_ends_with:
        #     continue

        # for not_contain in not_contains:
        #     if not_contain in node_path:
        #         node_not_contains = False

        # if node_not_contains:
        #     continue

        if keep_folder_structure:
            previous_path = ''
            for path in depth_path:
                id = f'{previous_path}/{path}' if previous_path != '' else path

                if not any(node['id'] == id for node in nodes):
                    nodes.append({
                        'id': id,
                        'label': path,
                    })
                
                if previous_path != '' and not any(link['source'] == previous_path and link['target'] == path for link in links):
                    links.append({
                        'source': previous_path,
                        'target': id,
                        'weight': 1,
                    })

                previous_path = path
        else:
            if not node in nodes:
                nodes.append({'id': '/'.join(depth_path)})

    for link in data['links']:
        source_node = link['source'].split('/')[:depth] if depth > 0 else link['source'].split('/')
        source_node_path = '/'.join(source_node)
        target_node_path = '/'.join(link['target'].split('/')[:depth] if depth > 0 else link['target'].split('/'))

        if source_node_path == target_node_path:
            continue

        if not any(temp['source'] == source_node_path and temp['target'] == target_node_path for temp in links):
            links.append({'source': source_node_path, 'target': target_node_path, 'weight': 1})
        else:
            for temp in links: 
                if temp['source'] == source_node_path and temp['target'] == target_node_path:
                    temp['weight'] = temp['weight'] + 1

    graph_data = {'nodes': nodes, 'links': links}

    net = Network(notebook=True, cdn_resources='in_line', select_menu=True, filter_menu=True, directed=True)

    for node in graph_data['nodes']:
        net.add_node(node['id'], label=node['id'])

    for link in graph_data['links']:
        if not any(node['id'] == link['source'] for node in net.nodes) or not any(node['id'] == link['target'] for node in net.nodes):
            continue

        net.add_edge(link['source'], link['target'], label=str(link['weight']), title=link['weight'])

    return net.generate_html()

# generate_graph(load_data(), 2, starts_with=['packages'])


## Web App

In [66]:
from IPython.display import display, HTML
from IPython.display import IFrame

# Display 'temp.html' in an iframe
depth = 3

with open('temp.html', 'w') as file:
    data = load_data()

    # file.write(generate_graph(load_data(), 1, not_starts_with=['examples', 'tests', 'docker', 'playwright-tests']))
    # file.write(generate_graph(load_data(), 2, starts_with=['apps', 'packages']))
    # file.write(generate_graph(load_data(), 2, starts_with=['packages']))
    # file.write(construct_network_graph(data['nodes'], data['links'], depth, starts_with=['apps', 'packages']))
    file.write(construct_network_graph(data['nodes'], data['links'], depth))

# display(HTML(generate_graph(load_data(), 1)))
# display(IFrame(src='./temp.html', width=700, height=600))
# HTML('./temp.html')